<a href="https://colab.research.google.com/github/kagankazim/FreshAir/blob/master/FreshairPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install geopandas


İlk olarak geopandas librarysini ekliyoruz

In [0]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import pandas as pd
import geopandas as gpd
import folium
import math
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [0]:
cred = credentials.Certificate('/content/drive/My Drive/ColabResource/freshair-491b9-firebase-adminsdk-9w0q4-76684634c1.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://freshair-491b9.firebaseio.com/'
})



In [0]:
ref = db.reference('airQuality')



In [12]:
dataframe1= pd.DataFrame(ref.get()[20:40])
dataframe1

,altitude,latitude,longitude,quality
0,0,38.552269,27.048168,40
1,0,38.552269,27.048168,40
2,0,38.552269,27.048168,40
3,0,38.552269,27.048168,40
4,0,38.552269,27.048168,40
5,0,38.552269,27.048168,40
6,0,38.552269,27.048168,40
7,0,38.552269,27.048168,40
8,0,38.552269,27.048168,40
9,0,38.552269,27.048168,40


In [13]:
dataframe1.dtypes

altitude       int64
latitude     float64
longitude    float64
quality        int64
dtype: object

In [0]:
m=folium.Map(location=[38.7996, 26.9707],tiles="openstreetmap",zoom_start=10)

In [0]:
for idx, row in dataframe1.iterrows():
    Marker([row['latitude'], row['longitude']]).add_to(m)
    


In [16]:
m